#### Исходные данные

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

#### Что сделано

Проведён анализ данных, сгруппированы объекты, выделены особые категории. Сформированы новые признаки. Подобрана оптимальная группировка. Удалены лишние признаки, ухудшающие качество работы модели. Подобраны параметры модели.

# 1. Подготовка данных

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
import timeit
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
df = pd.read_csv('/datasets/autos.csv')

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df['Price'].hist(bins=20, range=(0, 20000))

In [ ]:
df_notrepaired = df[df['NotRepaired'] == 'yes']
df_repaired = df[df['NotRepaired'] == 'no']
df_notrepaired_nan = df[df['NotRepaired'].isnull()]

In [ ]:
df_notrepaired_nan['Price'].hist()

In [ ]:
df_notrepaired['Price'].hist()

In [ ]:
df_repaired['Price'].hist()

In [ ]:
df_notrepaired_nan['Kilometer'].hist()

In [ ]:
df['Kilometer'].hist()

In [ ]:
df['RegistrationYear'].hist(bins=16, range=(1990, 2006))

In [ ]:
df_notrepaired_nan['RegistrationYear'].hist(bins=16, range=(1990, 2006))

In [ ]:
columns_name = df.columns

In [ ]:
for col in columns_name:
    print(col, df[col].unique())

In [ ]:
df_gby_model = df.groupby('Model').count()

In [ ]:
df_gby_model = df_gby_model['Price']

In [ ]:
tmp = df_gby_model.sort_values()
tmp

In [ ]:
model_min = tmp[:7]
model_min = model_min.index

In [ ]:
model_min

In [ ]:
tmp

In [ ]:
df_golf = df[df['Model'] == 'golf']

In [ ]:
df_golf['Price'].hist()

In [ ]:
df_golf.plot(x='Price', y='RegistrationYear')

In [ ]:
df_golf['RegistrationYear'].unique()

In [ ]:
def make_year (year):
    if year > 2016 or year < 1910:
        return np.nan
    elif year < 1996:
        return 22
    else:
        return 2017 - year
    
df['year'] = df.apply(lambda x: make_year(x['RegistrationYear']), axis=1)

In [ ]:
def make_model (model):
    if model in model_min:
        return np.nan
    else:
        return model
    
df['model'] = df.apply(lambda x: make_model(x['Model']), axis=1)

In [ ]:
def make_power (power):
    if power == 0 or power > 1000:
        return np.nan
    else:
        return power

df['power'] = df.apply(lambda x: make_power(x['Power']), axis=1)

In [ ]:
df.drop(['Power', 'Model', 'RegistrationYear', 'NumberOfPictures'], axis=1, inplace=True)

In [ ]:
tmp_list = [df, df_notrepaired, df_repaired, df_notrepaired_nan]
for df_i in tmp_list:
    df_i.plot(x='Price', y='Kilometer', kind='scatter', alpha=0.01)

In [ ]:
df['NotRepaired'] = df['NotRepaired'].fillna('yes')

In [ ]:
df_oldmobile = df[df['year'] == 22]
df_oldmobile.info()

In [ ]:
df_oldmobile['Price'].hist()

In [ ]:
df.plot(x='Price', y='year', kind='scatter', alpha=0.01, grid=True)

In [ ]:
def year_type (year):
    if year > 0 and year < 5:
        return 'new0_4'
    elif year >= 5 and year <= 12:
        return 'old5_12'
    elif year > 12 and year <= 17:
        return 'old13_17'
    elif year > 18 and year < 22:
        return 'old18_22'
    else:
        return year

df['Type_year'] = df.apply(lambda x: year_type(x['year']), axis=1)

In [ ]:
def year_type (mileage):
    if mileage > 140000:
        return 'yes'
    else:
        return 'no'

df['mileage_140'] = df.apply(lambda x: year_type(x['Kilometer']), axis=1)

In [ ]:
def group_by_brand (df, min_group, show_group=False):
    df_noold = df[df['year'] != 22] #выписываем автомобили моложе 22 лет.
    df_brend = df_noold.groupby(['Brand']).count()
    top_brend = df_brend[df_brend['Price'] > min_group]
    top_brend = top_brend['Price']
    all_brend = df_noold['Brand'].unique()
    list_top_brend = list(top_brend.index)
    df_any_grouped = []
    df_any_grouped.append(df_oldmobile)
    for brend in list_top_brend:
        df_tmp = df_noold[df_noold['Brand'] == brend]
        df_any_grouped.append(df_tmp)
        if df_tmp.shape[0] < min_group * 4 and show_group==True:
            print(brend)
            plt.figure()
            df_tmp.plot(x='Price', y='year', kind='scatter', alpha=0.5, grid=True)
    df_noname = df_noold.loc[~df_noold['Brand'].isin(list_top_brend)]
    df_any_grouped.append(df_noname)
    total = 0
    for df_by_brend in df_any_grouped:
        tmp_total = df_by_brend.shape[0]
        total += tmp_total
    if total - df.shape[0] == 0:
        print('База перераспределена без потерь')
    else:
        print('База перераспределена с потерями')
    return df_any_grouped, list_top_brend

In [ ]:
group_by_brand_result = group_by_brand(df, 5000, show_group=True)
df_grouped = group_by_brand_result[0]
list_top_brend = group_by_brand_result[1]

In [ ]:
def make_model1 (brand):
    if brand == 'fiat' or brand == 'mazda':
        return 'fiatmazda'
    elif brand == 'peugeot' or brand == 'renault':
        return 'peugeotrenault'
    elif brand == 'seat' or brand == 'skoda':
        return 'seatskoda'
    else:
        return brand
    
    
df['Brand'] = df.apply(lambda x: make_model1(x['Brand']), axis=1)
top_brand_del = ['fiat', 'mazda', 'peugeot','renault', 'seat', 'skoda']
top_brand_add = ['fiatmazda', 'peugeotrenault', 'seatskoda']
for brand_del in top_brand_del:
    list_top_brend.remove(brand_del)
for brand_add in top_brand_add:
    list_top_brend.append(brand_add)

In [ ]:
df_all = []
list_brand_all = []
for i in range(4):
    group_by_brand_result = group_by_brand(df, 5000 + i * 5000)
    df_grouped = group_by_brand_result[0]
    df_grouped.append(df) #Для проверки работающей модели на всей выборке добавим в группу всю выборку.
    for df_gr in df_grouped[1:-2]: #В таблицах сгруппированных по одному бренду - удаляем столбец "Brand".
        df_gr.drop(['Brand'], axis=1, inplace=True)
    list_top_brend = group_by_brand_result[1]
    df_all.append(df_grouped)
    list_brand_all.append(list_top_brend)

In [ ]:
df_all[2]

In [ ]:
col_for_del = set(['DateCrawled', 'VehicleType', 'RegistrationMonth', 'Brand', 'NotRepaired', 'DateCreated', 'PostalCode', 'LastSeen', 'Type_year', 'mileage_140'])

# 2. Обучение моделей

In [ ]:
cat_columns_all = set(['DateCrawled', 'VehicleType', 'Gearbox', 'FuelType', 'Brand', 'NotRepaired', 'DateCreated', 'LastSeen', 'model', 'Type_year', 'mileage_140'])

for i in range(len(df_all)):
    df_any_grouped = df_all[i]
    for df_gr in df_any_grouped:
        tmp_head_col = set(df_gr.columns)
        tmp_cat_col = cat_columns_all.intersection(tmp_head_col)
        tmp_cat_col = list(tmp_cat_col)
        for col in tmp_cat_col:
            df_gr[col] = df_gr[col].astype('category')

In [ ]:
def lgbm (df, num_leaves=50, learning_rate_default = 1, max_bin=50, num_iterations=100, time_calc=False):
    df_columns = set(df.columns)
    cat_columns = df_columns.intersection(cat_columns_all)
    cat_columns = list(cat_columns)
    train_features, test_features, train_target, test_target  = (
        train_test_split(df.drop(['Price'], axis=1), df['Price'], test_size=0.25, random_state =1))
    lgb_train = lgb.Dataset(train_features, train_target, categorical_feature=cat_columns)
    lgb_eval = lgb.Dataset(test_features, test_target, reference=lgb_train, categorical_feature=cat_columns)
    params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'rmse'},
    'num_leaves': num_leaves,
    'num_tress': 500,
    'num_iterations': num_iterations,
    'min_data_in_leaf' : 0,
    'min_sum_hessian_in_leaf' : 100,
    'max_depth' : -1,
    'num_threads': 16, 
    'max_bin' : max_bin,
    'learning_rate':  learning_rate_default,
    'feature_fraction': 0.5,
    'bagging_fraction': 1,
    'bagging_freq': 0
    }
    start_time_fit = timeit.default_timer()
    gbm = lgb.train(params,
    lgb_train,
    num_boost_round=10,
    valid_sets=lgb_eval,
    early_stopping_rounds=5)
    finish_time_fit = timeit.default_timer()
    time_fit = finish_time_fit - start_time_fit
    start_time_predict = timeit.default_timer()
    predictions = gbm.predict(test_features, num_iteration=gbm.best_iteration)
    finish_time_predict = timeit.default_timer()
    time_predict = finish_time_predict - start_time_predict
    if time_calc==True:
        return (test_target, predictions, time_fit, time_predict)
    rmse_level = mean_squared_error(predictions, test_target)**0.5
    return rmse_level

In [ ]:
def get_time (df_all, learning_rate=1, max_bin=50, return_time=False):
    predictions = []
    target = []
    time_fit1 = []
    time_predict1 = []
    for i in range(len(df_all)):
        score_for_glgbm = lgbm(df_all[i], best_config[i], learning_rate, max_bin, num_iterations=150, time_calc=True)
        tmp_target = pd.Series(score_for_glgbm[0])
        tmp_pred = pd.Series(score_for_glgbm[1])
        tmp_time_fit = score_for_glgbm[2]
        tmp_time_predict = score_for_glgbm[3]
        target.extend(tmp_target)
        predictions.extend(tmp_pred)
        time_fit1.append(tmp_time_fit)
        time_predict1.append(tmp_time_predict)
    rmse_level = mean_squared_error(predictions, target)**0.5
    if return_time==True:
        return (rmse_level, sum(time_fit1), sum(time_predict1) / len(df_all))
    else:
        return rmse_level

In [ ]:
best_config = [50] * 20 

In [ ]:
i = 0
tmp_result = []
for df_gr in df_all:
    i += 1
    tmp_rmse = get_time(df_gr, return_time=False)
    tmp_result.append(tmp_rmse)

In [ ]:
tmp_result

In [ ]:
col_for_del_year = ['year', 'Type_year']

In [ ]:
def check_category_rmse (df, list_col):
    tmp_result = []
    for col in list_col:
        tmp_rmse = lgbm(df.drop(col, axis=1))
        print('df-delcol', df.drop(col, axis=1).info())
        tmp_result.append(tmp_rmse)
        print('tmpRMSE', tmp_rmse, col)
    tmp_result.append(lgbm(df))
    return tmp_result

In [ ]:
tmp_result = check_category_rmse(df, col_for_del_year)
tmp_result

In [ ]:
tmp_result = check_category_rmse(df, col_for_del_mileage)
tmp_result

In [ ]:
col_for_del_mileage = ['Kilometer', 'mileage_140']

In [ ]:
df_any_grouped = df_all[0]

In [ ]:
df_all = [] # список где хранятся все базы данных, после удаления лишних столбцов
rmse_list = [] # список всех значений RMSE для каждой отдельной таблицы
col_for_del_proven = [] # список всех колонок на удаление для каждой отдельной таблицы
for df_gr in df_any_grouped:
    rmse_df = lgbm(df_gr)
    tmp_col_for_del_proven = []
    tmp_rmse_list = [rmse_df]
    df_min = df_gr.copy()
    tmp_col_for_del = df_min.columns.intersection(col_for_del)
    tmp_col_for_del = list(tmp_col_for_del)
    for col in tmp_col_for_del:
        print('Колонка тест', col)
        df_tmp = df_min.drop(col, axis=1)
        rmse_df_tmp = lgbm(df_tmp)
        if rmse_df_tmp < rmse_df:
            rmse_df = rmse_df_tmp
            tmp_col_for_del_proven.append(col)
            df_min = df_tmp
        tmp_rmse_list.append(rmse_df_tmp)
    df_all.append(df_min)
    rmse_list.append(tmp_rmse_list)
    col_for_del_proven.append(tmp_col_for_del_proven)

In [ ]:
col_for_del_proven

In [ ]:
best_rmse = []
for list_gr in rmse_list:
    best_gr_rmse = min(list_gr)
    best_rmse.append(best_gr_rmse)
best_rmse

In [ ]:
df_all[-1]

In [ ]:
def config_model (df, best_rmse):
    num_leavex_best = 50
    for i in (range(10)):
        tmp_num_leaves =  50 + i * 50
        tmp_rmse = lgbm(df, num_leaves=tmp_num_leaves)
        if best_rmse > tmp_rmse:
            best_rmse = tmp_rmse
            num_leavex_best = tmp_num_leaves
    return num_leavex_best

In [ ]:
best_config = []
for ii in range(len(df_all)):
    best_rmse_gr = best_rmse[ii]
    best_config_gr = config_model(df_all[ii], best_rmse_gr) # здесь хранится итог, и показатель num_leaves для лучшей точности
    best_config.append(best_config_gr)

best_config

In [ ]:
df_all[:-1]

In [ ]:
time_result = []
for i in range(5):
    tmp_time_result = get_time(df_all[:-1], learning_rate=0.5/(i+1), max_bin=50*(i+1), return_time=True)
    time_result.append(tmp_time_result)

In [ ]:
time_result_full = []
for i in range(5):
    tmp_time_result_full = lgbm(df_all[-1], learning_rate_default=0.5/(i+1), max_bin=50*(i+1), time_calc=True)
    rmse_level = mean_squared_error(tmp_time_result_full[1], tmp_time_result_full[0])**0.5
    tmp_time_fit = tmp_time_result_full[2]
    tmp_time_predict = tmp_time_result_full[3]
    time_result_full_tmp = [rmse_level, tmp_time_fit, tmp_time_predict]
    time_result_full.append(time_result_full_tmp)

# 3. Анализ моделей

In [ ]:
df_total = pd.DataFrame(time_result, columns=['rmse', 'time_fit', 'time_predict'])
df_total

In [ ]:
df_total_full = pd.DataFrame(time_result_full, columns=['rmse', 'time_fit', 'time_predict'])
df_total_full